In [4]:

import pandas as pd

files_path="/Users/vanshaj/Work/GitHub/Quant_Labs/Application/Data/Assets Data/EQUITY/INDIA"
symbol ="/Users/vanshaj/Work/GitHub/Quant_Labs/Application/Data/India50_stocks.parquet"


ds = pd.read_parquet(symbol)["Symbol"]

for i in ds:
     df = pd.read_parquet(f"{files_path}/{i}_history.parquet")
     print(df.head(1))



         Date    Open     High     Low   Close    Volume
0  2025-03-28  1280.0  1295.75  1269.0  1275.1  18147129
         Date    Open    High      Low   Close    Volume
0  2025-03-28  1829.0  1836.0  1805.85  1828.2  14386824
         Date    Open     High     Low    Close   Volume
0  2025-03-28  3651.0  3660.95  3591.9  3606.15  2051919
         Date    Open     High     Low   Close   Volume
0  2025-03-28  1733.3  1745.75  1660.0  1733.4  4133264
         Date    Open     High     Low    Close   Volume
0  2025-03-28  1336.0  1357.95  1327.2  1348.35  8266544
         Date   Open   High    Low  Close   Volume
0  2025-03-28  772.1  778.6  767.0  771.5  7628185
         Date    Open     High     Low    Close   Volume
0  2025-03-28  1590.0  1598.45  1560.0  1570.65  6799047
         Date    Open    High     Low    Close   Volume
0  2025-03-28  2247.0  2296.0  2240.3  2258.85  2280152
         Date   Open   High     Low   Close    Volume
0  2025-03-28  410.9  415.6  408.15  409.75  12981